In [6]:
import torch
from architectures import DANN
import pandas as pd
import numpy as np

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)

In [ ]:
model = torch.load("RBbot_old.pth", map_location=torch.device('cpu'))
model = model.eval()
model = model.to(device)

torch.save(model.state_dict(), 'RBbot_new.pth')

In [7]:
config = {
    "model_name": "DANN",
    "learning_rate": 1e-06,
    "source_batch_size": 64,
    "target_batch_size": 256,
    "epochs": 500,
    "image_size": 41,
    "random_seed": 2,
    "figs_every_epoch": 1,
    "num_domains": 2,
    "source_dataset_kind": "ZTF",
    "source_dataset_variant": "m6",
    "target_dataset_kind": "DECam",
    "target_dataset_variant": "m3",
    "conv_kernel": 3,
    "conv1_channels": 32,
    "conv1_dropout": 0.25,
    "conv2_channels": 32,
    "conv2_dropout": 0.25,
    "class1_neurons": 64,
    "class1_dropout": 0.25,
    "class2_neurons": 64,
    "class2_dropout": 0.25,
    "class3_neurons": 64,
    "class3_dropout": 0.25,
    "domain1_neurons": 64,
    "testing": 0,
    "run_name": "quiet-shadow-131"
}


In [8]:
def load_model(config):
    
    model = DANN(config)
    model.load_state_dict(torch.load('RBbot_new.pth'))
    model.eval()
    
    return model

In [9]:
model = load_model(config)
model = model.to(device)

In [ ]:
model = torch.load('RBbot_old.pth', map_location=torch.device('cpu'))
model = model.eval()
model = model.to(device)

In [ ]:
input_tensor = torch.randn(10, 3, 41, 41).to(device)  # Example input

raw_preds, _ = model(input_data=input_tensor, lamb=1)
preds = torch.round(raw_preds.detach())
print(preds)

In [ ]:
dataset_kind = config['source_dataset_kind']
dataset_variant = config['source_dataset_variant']
image_size = config['image_size']

import torchvision.transforms.v2 as transforms
from RBbot.DANN.utils import CustomDataset
from torch.utils.data import DataLoader
import RBbot.alert_utils

batch_size = config['target_batch_size']

triplets_path = f"/Users/nabeelr/Desktop/School/LS4/RBbot/RBbot/data/{dataset_kind}/val_triplets_{dataset_variant}.npy"
labels_path = f"/Users/nabeelr/Desktop/School/LS4/RBbot/RBbot/data/{dataset_kind}/val_cand_{dataset_variant}.csv"
labels = pd.read_csv(labels_path, index_col=None)['label']

triplets = np.load(triplets_path)
if np.shape(triplets)[-1] != image_size:
    triplets = RBbot.alert_utils.crop_triplets(triplets, image_size)
triplets = torch.from_numpy(triplets)

transform = transforms.Compose([
    transforms.ToDtype(torch.float32)
])

dataset = CustomDataset(
    data=triplets, labels=labels, transform=transform
)

dataloader = DataLoader(
    dataset=dataset, batch_size=batch_size, shuffle=True
)

data_target_iter = iter(dataloader)
len_dataloader = len(dataloader)

In [ ]:
batch_idx = 0
n_total = 0
n_correct = 0
all_raw_preds = np.array(())
all_labels = np.array(())

# For each batch...
while batch_idx < len_dataloader:
    # Get next batch of data
    batch_data = next(data_target_iter)
    trips, labels = batch_data
    trips = trips.to(device)
    labels = labels.to(device)

    # The last batch is a different size to make sure all examples are seen
    # Not necessary during training
    cur_batch_size = len(labels)

    # Run model
    raw_preds, _ = model(input_data=trips, lamb=1)

    # Convert raw scores to rb class predictions
    preds = torch.round(raw_preds.detach())

    # Compute number of correct examples
    n_correct += preds.eq(labels.detach().view_as(preds)).cpu().sum()
    n_total += cur_batch_size

    # Store quantities for diagnostics later
    all_raw_preds = np.concatenate((all_raw_preds, raw_preds.detach().cpu().numpy()))
    all_labels = np.concatenate((all_labels, labels.cpu().numpy()))

    batch_idx += 1

print(n_correct / n_total)


In [ ]:
trips = np.load("ZTF_exs.npy")

# Convert numpy array triplets to pytorch tensor and put them on the device
trips_tensor = torch.Tensor(trips).to(device)

# Run the model on the triplets
output = model(trips_tensor)

scores = output[0].detach().cpu().numpy()

print(scores)